In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from keras.layers.merge import concatenate
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [72]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [73]:
kospi = pd.read_csv('data/kospi200.csv', encoding='euc-kr')
kospi = kospi.sort_values(['일자'], ascending=[True])

kospi = kospi.iloc[:,1:].copy()

kospi = pd.concat([kospi.iloc[:,:3], kospi.iloc[:,4], kospi.iloc[:,3]], axis=1)

kospi['거래량'] = kospi['거래량'].apply(lambda x: x.replace(",","",-1))

for i in kospi.columns:
    kospi[i] = kospi[i].astype(float)

kospi = np.array(kospi)

In [74]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')
data = data.sort_values(['일자'], ascending=[True])

data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [75]:
n_steps = 5
samsung_x, samsung_y = split_sequence(data, n_steps)
kospi_x, _ = split_sequence(kospi, n_steps)

In [76]:
samsung_x = samsung_x.reshape((samsung_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(samsung_x)
samsung_x = scaler.transform(samsung_x)
print(samsung_x.shape)
kospi_x = kospi_x.reshape((kospi_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(kospi_x)
kospi_x = scaler.transform(kospi_x)
print(kospi_x.shape)

(421, 25)
(421, 25)


In [77]:
final_predict_samsung, final_predict_kospi = samsung_x[-1:,:], kospi_x[-1:,:]
from sklearn.model_selection import train_test_split

samsung_x_train, samsung_x_test, kospi_x_train, kospi_x_test, y_train, y_test = train_test_split(
    samsung_x, kospi_x, samsung_y, train_size=0.9, random_state=66, shuffle=True
)

In [78]:
inp1 = Input(shape=(samsung_x.shape[1],))
x = Dense(128, activation='relu', kernel_initializer='he_normal')(inp1)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
x = Dropout(0.25)(x)
out1 = Dense(32, activation="relu", kernel_initializer='he_normal')(x)

inp2 = Input(shape=(kospi_x.shape[1],))
x1 = Dense(128, activation='relu', kernel_initializer='he_normal')(inp2)
x = Dropout(0.25)(x)
x1 = Dense(64, activation='relu', kernel_initializer='he_normal')(x1)
x = Dropout(0.25)(x)
out2 = Dense(32, activation='relu', kernel_initializer='he_normal')(x1)

merge = concatenate([out1, out2])
merge = Dense(32, activation='relu', kernel_initializer='he_normal')(merge)
x = Dropout(0.25)(x)
merge = Dense(16, activation='relu', kernel_initializer='he_normal')(merge)
x = Dropout(0.25)(x)
output = Dense(1, activation='linear')(merge)

model = Model(inputs=[inp1, inp2], outputs=output)

In [79]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=50, mode='auto')
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit([samsung_x_train, kospi_x_train], y_train, epochs=2000,
         validation_split=0.1, callbacks=[early_stopping])

Train on 340 samples, validate on 38 samples
Epoch 1/2000
340/340 [==============================] - 1s 4ms/step - loss: 2201563965.7412 - acc: 0.0000e+00 - val_loss: 2198884291.3684 - val_acc: 0.0000e+00
Epoch 2/2000
340/340 [==============================] - 0s 65us/step - loss: 2200016516.5176 - acc: 0.0000e+00 - val_loss: 2196348288.0000 - val_acc: 0.0000e+00
Epoch 3/2000
340/340 [==============================] - 0s 59us/step - loss: 2196130331.1059 - acc: 0.0000e+00 - val_loss: 2189643580.6316 - val_acc: 0.0000e+00
Epoch 4/2000
340/340 [==============================] - 0s 56us/step - loss: 2186049486.3059 - acc: 0.0000e+00 - val_loss: 2171757460.2105 - val_acc: 0.0000e+00
Epoch 5/2000
340/340 [==============================] - 0s 62us/step - loss: 2159506573.5529 - acc: 0.0000e+00 - val_loss: 2125568559.1579 - val_acc: 0.0000e+00
Epoch 6/2000
340/340 [==============================] - 0s 62us/step - loss: 2092387558.4000 - acc: 0.0000e+00 - val_loss: 2013036651.7895 - val_acc: 0

340/340 [==============================] - 0s 56us/step - loss: 65232216.3294 - acc: 0.0000e+00 - val_loss: 92141939.1579 - val_acc: 0.0000e+00
Epoch 52/2000
340/340 [==============================] - 0s 62us/step - loss: 60314879.6941 - acc: 0.0000e+00 - val_loss: 86377303.7895 - val_acc: 0.0000e+00
Epoch 53/2000
340/340 [==============================] - 0s 61us/step - loss: 57058349.0824 - acc: 0.0000e+00 - val_loss: 81656522.3158 - val_acc: 0.0000e+00
Epoch 54/2000
340/340 [==============================] - 0s 56us/step - loss: 55110664.1412 - acc: 0.0000e+00 - val_loss: 79244393.2632 - val_acc: 0.0000e+00
Epoch 55/2000
340/340 [==============================] - 0s 64us/step - loss: 50578223.1529 - acc: 0.0000e+00 - val_loss: 75596255.1579 - val_acc: 0.0000e+00
Epoch 56/2000
340/340 [==============================] - 0s 65us/step - loss: 51076700.2588 - acc: 0.0000e+00 - val_loss: 71577025.8947 - val_acc: 0.0000e+00
Epoch 57/2000
340/340 [==============================] - 0s 62us/s

Epoch 103/2000
340/340 [==============================] - 0s 56us/step - loss: 10251107.6647 - acc: 0.0029 - val_loss: 19512142.3158 - val_acc: 0.0000e+00
Epoch 104/2000
340/340 [==============================] - 0s 56us/step - loss: 9888966.4000 - acc: 0.0000e+00 - val_loss: 19331011.8421 - val_acc: 0.0000e+00
Epoch 105/2000
340/340 [==============================] - ETA: 0s - loss: 7371591.0000 - acc: 0.0000e+0 - 0s 56us/step - loss: 9117641.0471 - acc: 0.0000e+00 - val_loss: 19671929.6316 - val_acc: 0.0000e+00
Epoch 106/2000
340/340 [==============================] - 0s 62us/step - loss: 8714845.4941 - acc: 0.0000e+00 - val_loss: 18502771.7895 - val_acc: 0.0000e+00
Epoch 107/2000
340/340 [==============================] - 0s 56us/step - loss: 9462370.0471 - acc: 0.0000e+00 - val_loss: 19518404.4737 - val_acc: 0.0000e+00
Epoch 108/2000
340/340 [==============================] - 0s 59us/step - loss: 9248515.1882 - acc: 0.0000e+00 - val_loss: 17513977.0526 - val_acc: 0.0000e+00
Epoch 1

Epoch 155/2000
340/340 [==============================] - 0s 56us/step - loss: 3206319.7824 - acc: 0.0000e+00 - val_loss: 8879796.4211 - val_acc: 0.0000e+00
Epoch 156/2000
340/340 [==============================] - 0s 53us/step - loss: 3096907.8706 - acc: 0.0000e+00 - val_loss: 8506949.1053 - val_acc: 0.0000e+00
Epoch 157/2000
340/340 [==============================] - 0s 56us/step - loss: 3040843.3324 - acc: 0.0000e+00 - val_loss: 8495808.8947 - val_acc: 0.0000e+00
Epoch 158/2000
340/340 [==============================] - 0s 59us/step - loss: 2851690.5882 - acc: 0.0000e+00 - val_loss: 8434717.0526 - val_acc: 0.0000e+00
Epoch 159/2000
340/340 [==============================] - 0s 53us/step - loss: 3195707.4824 - acc: 0.0000e+00 - val_loss: 8437524.1579 - val_acc: 0.0000e+00
Epoch 160/2000
340/340 [==============================] - 0s 59us/step - loss: 2917969.5324 - acc: 0.0029 - val_loss: 8236320.6842 - val_acc: 0.0000e+00
Epoch 161/2000
340/340 [==============================] - 0s 5

340/340 [==============================] - 0s 59us/step - loss: 1786122.2824 - acc: 0.0000e+00 - val_loss: 5883851.8947 - val_acc: 0.0000e+00
Epoch 208/2000
340/340 [==============================] - 0s 59us/step - loss: 1762860.6809 - acc: 0.0000e+00 - val_loss: 5772272.4737 - val_acc: 0.0000e+00
Epoch 209/2000
340/340 [==============================] - 0s 59us/step - loss: 1730320.9779 - acc: 0.0029 - val_loss: 5563822.1053 - val_acc: 0.0000e+00
Epoch 210/2000
340/340 [==============================] - 0s 65us/step - loss: 1608624.5794 - acc: 0.0000e+00 - val_loss: 6215569.0000 - val_acc: 0.0000e+00
Epoch 211/2000
340/340 [==============================] - 0s 59us/step - loss: 2066863.6765 - acc: 0.0000e+00 - val_loss: 5600327.7895 - val_acc: 0.0000e+00
Epoch 212/2000
340/340 [==============================] - 0s 56us/step - loss: 1675818.4074 - acc: 0.0029 - val_loss: 5868066.8947 - val_acc: 0.0000e+00
Epoch 213/2000
340/340 [==============================] - 0s 59us/step - loss: 20

...

Epoch 416/2000
340/340 [==============================] - 0s 56us/step - loss: 739854.8971 - acc: 0.0000e+00 - val_loss: 2876647.8947 - val_acc: 0.0000e+00
Epoch 417/2000
340/340 [==============================] - 0s 56us/step - loss: 940637.1029 - acc: 0.0000e+00 - val_loss: 2973578.9474 - val_acc: 0.0000e+00
Epoch 418/2000
340/340 [==============================] - 0s 62us/step - loss: 862386.5794 - acc: 0.0000e+00 - val_loss: 3208225.5000 - val_acc: 0.0000e+00
Epoch 419/2000
340/340 [==============================] - 0s 55us/step - loss: 860152.0647 - acc: 0.0000e+00 - val_loss: 2860852.1579 - val_acc: 0.0000e+00
Epoch 420/2000
340/340 [==============================] - 0s 62us/step - loss: 846557.0404 - acc: 0.0000e+00 - val_loss: 2866354.1579 - val_acc: 0.0000e+00
Epoch 421/2000
340/340 [==============================] - 0s 56us/step - loss: 751397.2588 - acc: 0.0000e+00 - val_loss: 3106771.4211 - val_acc: 0.0000e+00
Epoch 422/2000
340/340 [==============================] - 0s 69u

Epoch 469/2000
340/340 [==============================] - 0s 59us/step - loss: 752272.8059 - acc: 0.0000e+00 - val_loss: 2820150.3684 - val_acc: 0.0000e+00
Epoch 470/2000
340/340 [==============================] - 0s 65us/step - loss: 703831.1706 - acc: 0.0000e+00 - val_loss: 2673335.6579 - val_acc: 0.0000e+00
Epoch 471/2000
340/340 [==============================] - 0s 56us/step - loss: 771078.6857 - acc: 0.0000e+00 - val_loss: 2762731.9079 - val_acc: 0.0000e+00
Epoch 472/2000
340/340 [==============================] - 0s 62us/step - loss: 905538.3250 - acc: 0.0000e+00 - val_loss: 2640074.5526 - val_acc: 0.0000e+00
Epoch 473/2000
340/340 [==============================] - 0s 55us/step - loss: 872437.1618 - acc: 0.0000e+00 - val_loss: 2791493.2105 - val_acc: 0.0000e+00
Epoch 474/2000
340/340 [==============================] - 0s 58us/step - loss: 766125.3603 - acc: 0.0000e+00 - val_loss: 2767766.1974 - val_acc: 0.0000e+00
Epoch 475/2000
340/340 [==============================] - 0s 63u

Epoch 522/2000
340/340 [==============================] - 0s 56us/step - loss: 712719.3526 - acc: 0.0000e+00 - val_loss: 3130939.6579 - val_acc: 0.0000e+00
Epoch 523/2000
340/340 [==============================] - 0s 53us/step - loss: 651501.9044 - acc: 0.0000e+00 - val_loss: 2743709.9211 - val_acc: 0.0263
Epoch 524/2000
340/340 [==============================] - 0s 53us/step - loss: 600176.4081 - acc: 0.0000e+00 - val_loss: 2682261.4737 - val_acc: 0.0000e+00
Epoch 525/2000
340/340 [==============================] - 0s 64us/step - loss: 645507.8596 - acc: 0.0000e+00 - val_loss: 2759931.6316 - val_acc: 0.0000e+00
Epoch 526/2000
340/340 [==============================] - 0s 53us/step - loss: 643592.5647 - acc: 0.0029 - val_loss: 2677594.2632 - val_acc: 0.0000e+00
Epoch 527/2000
340/340 [==============================] - 0s 56us/step - loss: 747992.3331 - acc: 0.0029 - val_loss: 2951851.2895 - val_acc: 0.0000e+00
Epoch 528/2000
340/340 [==============================] - 0s 65us/step - los

Epoch 575/2000
340/340 [==============================] - 0s 62us/step - loss: 813563.1654 - acc: 0.0000e+00 - val_loss: 3024510.4868 - val_acc: 0.0000e+00
Epoch 576/2000
340/340 [==============================] - 0s 62us/step - loss: 743648.7202 - acc: 0.0000e+00 - val_loss: 2969362.2237 - val_acc: 0.0000e+00
Epoch 577/2000
340/340 [==============================] - 0s 62us/step - loss: 641651.2445 - acc: 0.0000e+00 - val_loss: 2926572.7632 - val_acc: 0.0000e+00
Epoch 578/2000
340/340 [==============================] - 0s 53us/step - loss: 708031.9015 - acc: 0.0000e+00 - val_loss: 2903028.5526 - val_acc: 0.0000e+00
Epoch 579/2000
340/340 [==============================] - 0s 56us/step - loss: 681466.1662 - acc: 0.0000e+00 - val_loss: 2609050.1579 - val_acc: 0.0000e+00
Epoch 580/2000
340/340 [==============================] - 0s 59us/step - loss: 647158.8404 - acc: 0.0000e+00 - val_loss: 2651399.6053 - val_acc: 0.0000e+00
Epoch 581/2000
340/340 [==============================] - 0s 56u

Epoch 628/2000
340/340 [==============================] - 0s 62us/step - loss: 652653.2287 - acc: 0.0000e+00 - val_loss: 2987393.0789 - val_acc: 0.0000e+00
Epoch 629/2000
340/340 [==============================] - 0s 53us/step - loss: 725749.5316 - acc: 0.0029 - val_loss: 3177289.9737 - val_acc: 0.0000e+00
Epoch 630/2000
340/340 [==============================] - 0s 62us/step - loss: 549936.2820 - acc: 0.0000e+00 - val_loss: 2874690.4474 - val_acc: 0.0000e+00
Epoch 631/2000
340/340 [==============================] - 0s 59us/step - loss: 453100.1919 - acc: 0.0029 - val_loss: 2671823.8947 - val_acc: 0.0000e+00
Epoch 632/2000
340/340 [==============================] - 0s 56us/step - loss: 479860.2077 - acc: 0.0000e+00 - val_loss: 2614199.0395 - val_acc: 0.0000e+00
Epoch 633/2000
340/340 [==============================] - 0s 53us/step - loss: 464157.2610 - acc: 0.0000e+00 - val_loss: 2562157.1711 - val_acc: 0.0000e+00
Epoch 634/2000
340/340 [==============================] - 0s 62us/step -

In [80]:
y_predict = model.predict([samsung_x_test, kospi_x_test])

In [81]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 1410.6536105951322


In [82]:
print(model.predict([final_predict_samsung, final_predict_kospi]))

[[56579.37]]
